In [ ]:
!pip install numpy soundfile


In [ ]:
import numpy as np
import soundfile as sf
import os


In [ ]:
def load_audio(pth):
    x, sr = sf.read(pth)
    return x, sr

def save_audio(x, sr, out_pth):
    sf.write(out_pth, x, sr)

# blend noise into clean speech at some SNR
def mix_snr(clean, noise, snr_val):
    if noise.ndim == 1 and clean.ndim == 2:
        noise = np.stack([noise] * clean.shape[1], axis=1)
    elif noise.ndim == 2 and clean.ndim == 1:
        clean = np.stack([clean] * noise.shape[1], axis=1)

    rms_c = np.sqrt(np.mean(clean ** 2))
    rms_n = np.sqrt(np.mean(noise ** 2))

    # scale noise to match target SNR
    scale = rms_c / (10 ** (snr_val / 20))
    noise = noise * (scale / rms_n)

    mixed = clean + noise
    return clean, noise, mixed

# toss noise onto a speech file at different SNRs
def apply_noise(speech_path, noise_paths, snr_list, out_dir):
    os.makedirs(out_dir, exist_ok=True)

    speech, sr = load_audio(speech_path)

    for n_path in noise_paths:
        noise, _ = load_audio(n_path)

        # loop it if it's shorter
        if len(noise) < len(speech):
            reps = int(np.ceil(len(speech) / len(noise)))
            noise = np.tile(noise, reps)[:len(speech)]
        else:
            noise = noise[:len(speech)]

        for snr in snr_list:
            c, n, mix = mix_snr(speech, noise, snr)

            speech_tag = os.path.basename(speech_path).split('.')[0]
            noise_tag = os.path.basename(n_path).split('.')[0]
            fname = f"{speech_tag}_w_{noise_tag}_{snr}dB.wav"

            out_path = os.path.join(out_dir, fname)
            save_audio(mix, sr, out_path)

            print(f"saved it -> {out_path}")

# paths and stuff
clean_path = '/content/drive/MyDrive/SongProcessing/CleanSongsVocals/familyAffair_vocals.wav'

noise_paths = [
    '/content/drive/MyDrive/SpeechProcessing/Noise/AirConditioner_1.wav',
    '/content/drive/MyDrive/SpeechProcessing/Noise/Babble_8.wav',
    '/content/drive/MyDrive/SpeechProcessing/Noise/Munching_4.wav',
    '/content/drive/MyDrive/SpeechProcessing/Noise/Typing_3.wav',
    '/content/drive/MyDrive/SpeechProcessing/Noise/VacuumCleaner_3.wav'
]

snrs = [30, 20, 10, 5, 0]

out_folder = '/content/drive/MyDrive/SongProcessing/familyAffair_Noisy'

# run
apply_noise(clean_path, noise_paths, snrs, out_folder)
